# Character-level Language Model

Language Model의 목표는 입력 문서와 비슷한 새로운 텍스트를 생성하는 모델을 개발하는 것이다. 

문자 단위의 모델링에서 입력은 글자의 시퀀스를 나누어 한 번에 글자 하나씩 입력된다. 모형은 이 전 입력된 글자와 함께 새로 입력된 글자를 처리하여 다음 문자를 예측한다. 

![IMG](../assets/text_generator.png)

# 필요 라이브러리 import

GPU를 활성화하면 훈련 속도를 높일 수 있다.

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

import numpy as np
import os
import time

2.11.0



# 데이터 전처리

다음의 링크에서 세익스피어의 비극 텍스트 파일을 다운로드 하여 텍스트를 적절히 전처리 한다.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 1115394자


In [4]:
# 텍스트의 처음 250자를 살펴봅니다
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 65개


### 텍스트 벡터화

읽어들인 텍스트의 개별 문자들을 숫자로 변환한다.

* char2idx : 문자를 숫자로 매핑
* idx2char : 숫자를 문자로 매핑

In [37]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
print(char2idx)
print(idx2char)
print(text[:100])
print(text_as_int)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
[18 47 56 ... 45  8  0]


In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [41]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} -------- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' -------- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### 훈련 데이터 생성

입력 데이터 : 텍스트를 `seq_length`로 나눈다.

타깃 데이터 : 각 입력 시퀀스에서, 해당 타깃은 한 문자를 오른쪽으로 이동하여 동일한 길이의 텍스트로 생성한다.

 예를 들어, `seq_length`는 4이고 텍스트를 "Hello"이라고 가정해 봅시다. 입력 시퀀스는 "Hell"이고 타깃 시퀀스는 "ello"가 된다.

이렇게 하기 위해 먼저 `tf.data.Dataset.from_tensor_slices` 함수를 사용해 텍스트 벡터를 문자 인덱스의 스트림으로 변환한다.

In [64]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기
print(text_as_int[:10])
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
print(char_dataset)
for elem in char_dataset.take(10):
    print(elem.numpy(),' ', end='')
print('')

for i in char_dataset.take(5):
    print(idx2char[i.numpy()],'   ', i ,'   ', i.numpy())

[18 47 56 57 58  1 15 47 58 47]
<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
18  47  56  57  58  1  15  47  58  47  
F     tf.Tensor(18, shape=(), dtype=int32)     18
i     tf.Tensor(47, shape=(), dtype=int32)     47
r     tf.Tensor(56, shape=(), dtype=int32)     56
s     tf.Tensor(57, shape=(), dtype=int32)     57
t     tf.Tensor(58, shape=(), dtype=int32)     58


`batch` 메서드는 이 개별 문자들을 원하는 크기의 시퀀스로 쉽게 변환

In [68]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
print(sequences)

for item in sequences.take(5):
    print(item.numpy(),len(item.numpy()))

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int32, name=None)>
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1] 101
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49] 101
[52 53 61  1 15 39 47 59 57  1 25 39 56 41 47 59 57  1 47 57  1 41 46 47
 43 44  1 43 52 43 51 63  1 58 53  1 58 46 43  1 54 43 53 54 50 43  8  0
  0 13 50 50 10  0 35 43  1 49 52 53 61  5 58  6  1 61 43  1 49 52 53 61
  5 58  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 24 43 58  1
 5

각 시퀀스에서, `map` 메서드를 사용해 각 배치에 간단한 함수를 적용하고 입력 텍스트와 타깃 텍스트를 복사 및 이동:

In [69]:
def split_input_target(chunk):
    print(chunk)
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
print(dataset)

Tensor("args_0:0", shape=(101,), dtype=int32)
<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>


In [12]:
for input_example, target_example in  dataset.take(1):
    print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
타깃 데이터:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


이 벡터의 각 인덱스는 하나의 타임 스텝(time step)으로 처리된다. 타임 스텝 0의 입력으로 모델은 "F"의 인덱스를 받고 다음 문자로 "i"의 인덱스를 예측한다. 다음 타임 스텝에서도 같은 일을 하지만 RNN은 현재 입력 문자 외에 이전 타임 스텝을 고려한다.

In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 18 ('F')
  예상 출력: 47 ('i')
   1단계
  입력: 47 ('i')
  예상 출력: 56 ('r')
   2단계
  입력: 56 ('r')
  예상 출력: 57 ('s')
   3단계
  입력: 57 ('s')
  예상 출력: 58 ('t')
   4단계
  입력: 58 ('t')
  예상 출력: 1 (' ')


### 훈련 배치 생성

텍스트를 다루기 쉬운 시퀀스로 분리하기 위해 `tf.data`를 사용한다.

In [71]:
# 배치 크기
BATCH_SIZE = 64

BUFFER_SIZE = 10000

print(dataset)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(64, 64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 64, 100), dtype=tf.int32, name=None))>


In [75]:
list(dataset.take(1))

[(<tf.Tensor: shape=(64, 64, 100), dtype=int32, numpy=
  array([[[18, 47, 56, ..., 37, 53, 59],
          [39, 56, 43, ..., 53, 59,  1],
          [52, 53, 61, ..., 57,  1, 49],
          ...,
          [ 0, 35, 46, ..., 49,  1, 51],
          [ 6,  7,  7, ..., 60, 43, 56],
          [53, 59, 58, ..., 43, 39, 60]],
  
         [[ 1, 51, 43, ..., 26, 21, 33],
          [10,  0, 32, ..., 43, 50, 57],
          [39, 52, 42, ..., 40, 43, 52],
          ...,
          [32, 53,  1, ..., 56,  0, 20],
          [61,  1, 58, ...,  1, 39, 41],
          [47, 53, 52, ..., 58, 43, 56]],
  
         [[42,  1, 47, ..., 58,  1, 53],
          [ 1, 47, 52, ..., 57,  1, 45],
          [52, 43,  0, ..., 46, 43, 63],
          ...,
          [ 1, 58, 46, ..., 40, 43, 44],
          [56, 43,  1, ..., 51, 47, 52],
          [ 1, 46, 53, ..., 43, 56, 63]],
  
         ...,
  
         [[ 1, 58, 53, ..., 15, 20, 13],
          [16,  1, 21, ..., 52, 45,  1],
          [46, 39, 50, ..., 53, 56,  1],
          

## 모델 설계

모델을 정의하려면 `tf.keras.Sequential`을 사용한다. 이 간단한 예제에서는 3개의 층을 사용하여 모델을 정의:

* `tf.keras.layers.Embedding` : 입력층. `embedding_dim` 차원 벡터에 각 문자의 정수 코드를 매핑하는 훈련 가능한 검색 테이블.
* `tf.keras.layers.GRU` : 크기가 `units = rnn_units`인 RNN의 유형(여기서 LSTM층을 사용할 수도 있다.)
* `tf.keras.layers.Dense` : 크기가 `vocab_size`인 출력을 생성하는 출력층.

In [16]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## 모델 사용

이제 모델을 실행하여 원하는대로 동작하는지 확인한다.

먼저 출력의 형태를 확인한다:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(64, 100, 65) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


위 예제에서 입력의 시퀀스 길이는 100이지만 모델은 임의 길이의 입력에서 실행될 수 있다.

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


모델로부터 예측 문자열을 얻으려면 출력 배열에서 샘플링하여 실제 문자 인덱스를 얻어야 한다.

Note: 출력 배열에 argmax를 취하면 안되고 임의의 샘플링하는 것이 중요하다.

일단 배치의 첫 번째를 샘플링 하는 형태로 출력해 본다:

In [78]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# print(sampled_indices)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices)

[53  8  9 54  9 38 50 60 17 53 56 41 15 29 46 25 42  6 57 44 55 47 61 62
 52 61 16 37 14 17 25 26  2 61 24 14  9 10 61 40 35 35 51  3 29 64  5 23
 54 33 46 45 46 54  2 58 14 23 20 34 43 12 14  6 62 11 35  4 48 63 30  1
 57 11 42  2  3 43 45 60 55 16 20  9 35 63 29 51 30 38  5  9 15 38 42 18
 27 36 25 52]


In [22]:
sampled_indices

array([34, 63, 25, 21,  8, 48, 42, 15, 33, 49, 33, 39, 19, 41, 42, 55, 26,
       54, 33, 35,  7, 37,  2, 42, 37,  2, 64, 19, 33, 33, 10, 30, 55, 30,
       39, 38, 16, 23, 35, 57,  2, 62, 28, 22, 10,  4, 52, 24, 21, 64, 21,
        9, 63,  9, 42, 26, 10, 41, 43,  7, 29, 54, 61, 58, 43, 21, 37, 57,
       59, 28, 12, 53, 17, 34, 60, 46, 52,  6, 12, 52,  0, 25, 11, 54, 23,
       27,  1, 17, 18, 44, 16, 17, 41, 54, 51,  1, 17, 29, 49,  1],
      dtype=int64)

훈련되지 않은 모델에 의해 예측된 텍스트를 보기 위해 idx2char을 사용한다.


In [23]:
print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

예측된 다음 문자: 
 'VyMI.jdCUkUaGcdqNpUW-Y!dY!zGUU:RqRaZDKWs!xPJ:&nLIzI3y3dN:ce-QpwteIYsuP?oEVvhn,?n\nM;pKO EFfDEcpm EQk '


## 모델 훈련

### 모형의 compile과 loss 함수 정의

표준 `tf.keras.losses.sparse_softmax_crossentropy` 손실 함수를 사용한다.

이 모델은 로짓을 반환하기 때문에 `from_logits` 을 사용한다.

In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("Loss : ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 65)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
Loss :  4.174923


In [25]:
model.compile(optimizer='adam', loss=loss)

### 훈련 실행

훈련 시간이 너무 길지 않도록 모델을 훈련하는 데 에포크(Epoch)를 적절히 사용한다. 

In [26]:
EPOCHS=1

In [27]:
history = model.fit(dataset, epochs=EPOCHS)
#history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

172/172 [==============================] - 479s 3s/step - loss: 2.6362


In [28]:
# 모형을 저장한다.
model.save_weights('epoch_1.h5')

## 텍스트 생성

이 예측 단계를 간단하게 하기 위해 배치 크기로 1을 사용한다.

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# 저장된 모형을 불러들인다.
model.load_weights('epoch_20.h5')
model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 lstm_1 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### 텍스트 생성하기

다음 코드 블록은 텍스트를 생성합니다:

* 시작 문자열 선택과 순환 신경망 상태를 초기화하고 생성할 문자 수를 설정한다.

* 시작 문자열과 순환 신경망 상태를 사용하여 다음 문자의 예측 배열을 가져온다.

* 다음, 범주형 배열을 사용하여 예측된 문자의 인덱스를 계산합니다. 이 예측된 문자를 모델의 다음 입력으로 활용한다.

![IMG](https://tensorflow.org/tutorials/text/images/text_generation_sampling.png)



In [106]:
def generate_text(model, start_string):
  # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

  # 생성할 문자의 수
    num_generate = 1000

  # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = [char2idx[s] for s in start_string]
    print(input_eval)
    input_eval = tf.expand_dims(input_eval, 0)
    print(input_eval)
    
  # 결과를 저장할 빈 문자열
    text_generated = []

  # temperature 가 낮으면 더 예측 가능한 텍스트가 됩니다.
  # temperature 가 높으면 더 의외의 텍스트가 됩니다.
    temperature = 1.0

  # 여기에서 배치 크기 == 1
    print(model)
    model.reset_states()
    print(model)
    for i in range(num_generate):
        predictions = model(input_eval)
        print(predictions.shape,' ',end='')
      # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)
        print(predictions.shape,' ',end='')
      # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
#         print(tf.random.categorical(predictions, num_samples=1),' ',end='')
        print(predictions.shape, predicted_id,' ',end='')
        
      # 예측된 단어를 다음 입력으로 모델에 전달    
        input_eval = tf.expand_dims([predicted_id], 0)
        print(' input_eval:',input_eval,' ',end='')
        print(idx2char[predicted_id])
        text_generated.append(idx2char[predicted_id])
    print('')
    return (start_string + ''.join(text_generated))

In [107]:
print(generate_text(model, start_string="ROMEO: "))

[30, 27, 25, 17, 27, 10, 1]
tf.Tensor([[30 27 25 17 27 10  1]], shape=(1, 7), dtype=int32)
(1, 7, 65)  (7, 65)  (7, 65) 44   input_eval: tf.Tensor([[44]], shape=(1, 1), dtype=int32)  f
(1, 1, 65)  (1, 65)  (1, 65) 47   input_eval: tf.Tensor([[47]], shape=(1, 1), dtype=int32)  i
(1, 1, 65)  (1, 65)  (1, 65) 56   input_eval: tf.Tensor([[56]], shape=(1, 1), dtype=int32)  r
(1, 1, 65)  (1, 65)  (1, 65) 57   input_eval: tf.Tensor([[57]], shape=(1, 1), dtype=int32)  s
(1, 1, 65)  (1, 65)  (1, 65) 58   input_eval: tf.Tensor([[58]], shape=(1, 1), dtype=int32)  t
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 44   input_eval: tf.Tensor([[44]], shape=(1, 1), dtype=int32)  f
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 22   input_eval: tf.Tensor([[22]], sha

(1, 1, 65)  (1, 65)  (1, 65) 8   input_eval: tf.Tensor([[8]], shape=(1, 1), dtype=int32)  .
(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shape=(1, 1), dtype=int32)  

(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shape=(1, 1), dtype=int32)  

(1, 1, 65)  (1, 65)  (1, 65) 19   input_eval: tf.Tensor([[19]], shape=(1, 1), dtype=int32)  G
(1, 1, 65)  (1, 65)  (1, 65) 30   input_eval: tf.Tensor([[30]], shape=(1, 1), dtype=int32)  R
(1, 1, 65)  (1, 65)  (1, 65) 33   input_eval: tf.Tensor([[33]], shape=(1, 1), dtype=int32)  U
(1, 1, 65)  (1, 65)  (1, 65) 25   input_eval: tf.Tensor([[25]], shape=(1, 1), dtype=int32)  M
(1, 1, 65)  (1, 65)  (1, 65) 21   input_eval: tf.Tensor([[21]], shape=(1, 1), dtype=int32)  I
(1, 1, 65)  (1, 65)  (1, 65) 27   input_eval: tf.Tensor([[27]], shape=(1, 1), dtype=int32)  O
(1, 1, 65)  (1, 65)  (1, 65) 10   input_eval: tf.Tensor([[10]], shape=(1, 1), dtype=int32)  :
(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shap

(1, 1, 65)  (1, 65)  (1, 65) 56   input_eval: tf.Tensor([[56]], shape=(1, 1), dtype=int32)  r
(1, 1, 65)  (1, 65)  (1, 65) 42   input_eval: tf.Tensor([[42]], shape=(1, 1), dtype=int32)  d
(1, 1, 65)  (1, 65)  (1, 65) 12   input_eval: tf.Tensor([[12]], shape=(1, 1), dtype=int32)  ?
(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shape=(1, 1), dtype=int32)  

(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shape=(1, 1), dtype=int32)  

(1, 1, 65)  (1, 65)  (1, 65) 28   input_eval: tf.Tensor([[28]], shape=(1, 1), dtype=int32)  P
(1, 1, 65)  (1, 65)  (1, 65) 30   input_eval: tf.Tensor([[30]], shape=(1, 1), dtype=int32)  R
(1, 1, 65)  (1, 65)  (1, 65) 21   input_eval: tf.Tensor([[21]], shape=(1, 1), dtype=int32)  I
(1, 1, 65)  (1, 65)  (1, 65) 37   input_eval: tf.Tensor([[37]], shape=(1, 1), dtype=int32)  Y
(1, 1, 65)  (1, 65)  (1, 65) 10   input_eval: tf.Tensor([[10]], shape=(1, 1), dtype=int32)  :
(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], sh

(1, 1, 65)  (1, 65)  (1, 65) 21   input_eval: tf.Tensor([[21]], shape=(1, 1), dtype=int32)  I
(1, 1, 65)  (1, 65)  (1, 65) 5   input_eval: tf.Tensor([[5]], shape=(1, 1), dtype=int32)  '
(1, 1, 65)  (1, 65)  (1, 65) 50   input_eval: tf.Tensor([[50]], shape=(1, 1), dtype=int32)  l
(1, 1, 65)  (1, 65)  (1, 65) 50   input_eval: tf.Tensor([[50]], shape=(1, 1), dtype=int32)  l
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 44   input_eval: tf.Tensor([[44]], shape=(1, 1), dtype=int32)  f
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 50   input_eval: tf.Tensor([[50]], shape=(1, 1), dtype=int32)  l
(1, 1, 65)  (1, 65)  (1, 65) 50   input_eval: tf.Tensor([[50]], shape=(1, 1), dtype=int32)  l
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 61   input_eval: tf.Tensor([[61]], 

(1, 1, 65)  (1, 65)  (1, 65) 39   input_eval: tf.Tensor([[39]], shape=(1, 1), dtype=int32)  a
(1, 1, 65)  (1, 65)  (1, 65) 51   input_eval: tf.Tensor([[51]], shape=(1, 1), dtype=int32)  m
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 39   input_eval: tf.Tensor([[39]], shape=(1, 1), dtype=int32)  a
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 45   input_eval: tf.Tensor([[45]], shape=(1, 1), dtype=int32)  g
(1, 1, 65)  (1, 65)  (1, 65) 43   input_eval: tf.Tensor([[43]], shape=(1, 1), dtype=int32)  e
(1, 1, 65)  (1, 65)  (1, 65) 52   input_eval: tf.Tensor([[52]], shape=(1, 1), dtype=int32)  n
(1, 1, 65)  (1, 65)  (1, 65) 58   input_eval: tf.Tensor([[58]], shape=(1, 1), dtype=int32)  t
(1, 1, 65)  (1, 65)  (1, 65) 50   input_eval: tf.Tensor([[50]], shape=(1, 1), dtype=int32)  l
(1, 1, 65)  (1, 65)  (1, 65) 43   input_eval: tf.Tensor([[43]], 

...

(1, 1, 65)  (1, 65)  (1, 65) 60   input_eval: tf.Tensor([[60]], shape=(1, 1), dtype=int32)  v
(1, 1, 65)  (1, 65)  (1, 65) 43   input_eval: tf.Tensor([[43]], shape=(1, 1), dtype=int32)  e
(1, 1, 65)  (1, 65)  (1, 65) 6   input_eval: tf.Tensor([[6]], shape=(1, 1), dtype=int32)  ,
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 63   input_eval: tf.Tensor([[63]], shape=(1, 1), dtype=int32)  y
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 57   input_eval: tf.Tensor([[57]], shape=(1, 1), dtype=int32)  s
(1, 1, 65)  (1, 65)  (1, 65) 47   input_eval: tf.Tensor([[47]], shape=(1, 1), dtype=int32)  i
(1, 1, 65)  (1, 65)  (1, 65) 52   input_eval: tf.Tensor([[52]], shape=(1, 1), dtype=int32)  n
(1, 1, 65)  (1, 65)  (1, 65) 41   input_eval: tf.Tensor([[41]], shape=(1, 1), dtype=int32)  c
(1, 1, 65)  (1, 65)  (1, 65) 43   input_eval: tf.Tensor([[43]], sh

(1, 1, 65)  (1, 65)  (1, 65) 58   input_eval: tf.Tensor([[58]], shape=(1, 1), dtype=int32)  t
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 57   input_eval: tf.Tensor([[57]], shape=(1, 1), dtype=int32)  s
(1, 1, 65)  (1, 65)  (1, 65) 54   input_eval: tf.Tensor([[54]], shape=(1, 1), dtype=int32)  p
(1, 1, 65)  (1, 65)  (1, 65) 43   input_eval: tf.Tensor([[43]], shape=(1, 1), dtype=int32)  e
(1, 1, 65)  (1, 65)  (1, 65) 39   input_eval: tf.Tensor([[39]], shape=(1, 1), dtype=int32)  a
(1, 1, 65)  (1, 65)  (1, 65) 49   input_eval: tf.Tensor([[49]], shape=(1, 1), dtype=int32)  k
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 52   input_eval: tf.Tensor([[52]], shape=(1, 1), dtype=int32)  n
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], sh

(1, 1, 65)  (1, 65)  (1, 65) 45   input_eval: tf.Tensor([[45]], shape=(1, 1), dtype=int32)  g
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 52   input_eval: tf.Tensor([[52]], shape=(1, 1), dtype=int32)  n
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 58   input_eval: tf.Tensor([[58]], shape=(1, 1), dtype=int32)  t
(1, 1, 65)  (1, 65)  (1, 65) 12   input_eval: tf.Tensor([[12]], shape=(1, 1), dtype=int32)  ?
(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shape=(1, 1), dtype=int32)  

(1, 1, 65)  (1, 65)  (1, 65) 31   input_eval: tf.Tensor([[31]], shape=(1, 1), dtype=int32)  S
(1, 1, 65)  (1, 65)  (1, 65) 47   input_eval: tf.Tensor([[47]], shape=(1, 1), dtype=int32)  i
(1, 1, 65)  (1, 65)  (1, 65) 56   input_eval: tf.Tensor([[56]], shape=(1, 1), dtype=int32)  r
(1, 1, 65)  (1, 65)  (1, 65) 56   input_eval: tf.Tensor([[56]], 

(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 51   input_eval: tf.Tensor([[51]], shape=(1, 1), dtype=int32)  m
(1, 1, 65)  (1, 65)  (1, 65) 63   input_eval: tf.Tensor([[63]], shape=(1, 1), dtype=int32)  y
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 42   input_eval: tf.Tensor([[42]], shape=(1, 1), dtype=int32)  d
(1, 1, 65)  (1, 65)  (1, 65) 39   input_eval: tf.Tensor([[39]], shape=(1, 1), dtype=int32)  a
(1, 1, 65)  (1, 65)  (1, 65) 51   input_eval: tf.Tensor([[51]], shape=(1, 1), dtype=int32)  m
(1, 1, 65)  (1, 65)  (1, 65) 0   input_eval: tf.Tensor([[0]], shape=(1, 1), dtype=int32)  

(1, 1, 65)  (1, 65)  (1, 65) 24   input_eval: tf.Tensor([[24]], shape=(1, 1), dtype=int32)  L
(1, 1, 65)  (1, 65)  (1, 65) 47   input_eval: tf.Tensor([[47]], shape=(1, 1), dtype=int32)  i
(1, 1, 65)  (1, 65)  (1, 65) 60   input_eval: tf.Tensor([[60]], sh

(1, 1, 65)  (1, 65)  (1, 65) 44   input_eval: tf.Tensor([[44]], shape=(1, 1), dtype=int32)  f
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 56   input_eval: tf.Tensor([[56]], shape=(1, 1), dtype=int32)  r
(1, 1, 65)  (1, 65)  (1, 65) 45   input_eval: tf.Tensor([[45]], shape=(1, 1), dtype=int32)  g
(1, 1, 65)  (1, 65)  (1, 65) 53   input_eval: tf.Tensor([[53]], shape=(1, 1), dtype=int32)  o
(1, 1, 65)  (1, 65)  (1, 65) 58   input_eval: tf.Tensor([[58]], shape=(1, 1), dtype=int32)  t
(1, 1, 65)  (1, 65)  (1, 65) 1   input_eval: tf.Tensor([[1]], shape=(1, 1), dtype=int32)   
(1, 1, 65)  (1, 65)  (1, 65) 57   input_eval: tf.Tensor([[57]], shape=(1, 1), dtype=int32)  s
(1, 1, 65)  (1, 65)  (1, 65) 43   input_eval: tf.Tensor([[43]], shape=(1, 1), dtype=int32)  e
(1, 1, 65)  (1, 65)  (1, 65) 56   input_eval: tf.Tensor([[56]], shape=(1, 1), dtype=int32)  r
(1, 1, 65)  (1, 65)  (1, 65) 8   input_eval: tf.Tensor([[8]], 

# 실습

1. 시작 문자열을 다른 것으로 해보면 어떤 결과가 나오는가?

2.  generate_text 함수에서 temperature 값을 10으로 변경하여 생성되는 결과를 확인해 보시오.

3. 모형의 정확도를 높이기 위해 모형에 layer를 추가해 보시오.